# Detecting spouse mentions in sentences

In this tutorial, we will see how Snorkel can be used for Information Extraction. We will walk through an example text classification task for information extraction, where we use labeling functions involving keywords and distant supervision.

* Classification Task *
<img src="imgs/sentence.jpg" width="700px;" onerror="this.onerror=null; this.src='/doks-theme/assets/images/sentence.jpg';" align="center" style="display: block; margin-left: auto; margin-right: auto;">

We want to classify each __candidate__ or pair of people mentioned in a sentence, as being married at some point or not.

In the above example, our candidate represents the possible relation `(Barack Obama, Michelle Obama)`. As readers, we know this mention is true due to external knowledge and the keyword of `wedding` occuring later in the sentence.
We begin with some basic setup and data downloading.


In [1]:
%matplotlib inline

import os
import pandas as pd
import pickle

## Load data

In [2]:
from utils import load_data

((df_dev, Y_dev), df_train, (df_test, Y_test)) = load_data()

**Input Data:** `df_dev`, `df_train`, and `df_test` are `Pandas DataFrame` objects, where each row represents a particular __candidate__. For our problem, a candidate consists of a sentence, and two people mentioned in the sentence. The DataFrames contain the fields `sentence`, which refers to the sentence of the candidate, `tokens`, the tokenized form of the sentence, and `person1_word_idx` and `person2_word_idx`, which represent `[start, end]` indices in the tokens at which the first and second person's name appear, respectively.

We also have certain **preprocessed fields**, that we discuss a few cells below.

In [3]:
# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", 0)

df_dev.head(5)

,person1_word_idx,person2_word_idx,sentence,tokens,person1_right_tokens,person2_right_tokens,between_tokens
0,"(1, 1)","(22, 24)","The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.","[The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]","[are, half, -, sisters]","[., ]","[are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate]"
1,"(1, 1)","(7, 8)","The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.","[The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]","[are, half, -, sisters]","[,, the, mother, of]","[are, half, -, sisters, to]"
2,"(7, 8)","(22, 24)","The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.","[The, Richards, are, half, -, sisters, to, Kathy, Hilton, ,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate, Richard, Howard, Hilton, ., ]","[,, the, mother, of]","[., ]","[,, the, mother, of, socialite, Paris, Hilton, and, spouse, of, luxury, hotel, magnate]"
3,"(6, 6)","(20, 21)","Prior to both his guests, Colbert's monologue - parts of which he did sitting down - ripped into Donald Trump and his oft-mocked policy of building a wall at the US-Mexico border and not eating Oreos anymore.","[Prior, to, both, his, guests, ,, Colbert, s, monologue, -, parts, of, which, he, did, sitting, down, -, ripped, into, Donald, Trump, and, his, oft, -, mocked, policy, of, building, a, wall, at, the, US, -, Mexico, border, and, not, eating, Oreos, anymore, ., ]","[s, monologue, -, parts]","[and, his, oft, -]","[s, monologue, -, parts, of, which, he, did, sitting, down, -, ripped, into]"
4,"(2, 2)","(4, 5)","People reported Williams and Ven Veen tied the knot Saturday at Brush Creek Ranch in Saratoga, Wyoming, in front of about 200 guests.","[People, reported, Williams, and, Ven, Veen, tied, the, knot, Saturday, at, Brush, Creek, Ranch, in, Saratoga, ,, Wyoming, ,, in, front, of, about, 200, guests, .]","[and, Ven, Veen, tied]","[tied, the, knot, Saturday]",[and]


Let's look at a candidate in the development set:

In [4]:
from preprocessors import get_person_text

candidate = df_dev.loc[2]
person_names = get_person_text(candidate).person_names

print("Sentence: ", candidate["sentence"])
print("Person 1: ", person_names[0])
print("Person 2: ", person_names[1])

Sentence:  The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.   
Person 1:  Kathy Hilton
Person 2:  Richard Howard Hilton


### Preprocessing the Data

In a real application, there is a lot of data preparation, parsing, and database loading that needs to be completed before we generate candidates and dive into writing labeling functions. Here we've pre-generated candidates in a pandas DataFrame object per split (train,dev,test).

### Labeling Function Helpers

When writing labeling functions, there are several functions you will use over and over again. In the case of text relation extraction as with this task, common functions include those for fetching text between mentions of the two people in a candidate, examing word windows around person mentions, and so on. We will wrap these functions as `preprocessors`.

In [5]:
from snorkel.preprocess import preprocessor


@preprocessor()
def get_text_between(cand):
    """
    Returns the text between the two person mentions in the sentence for a candidate
    """
    start = cand.person1_word_idx[1] + 1
    end = cand.person2_word_idx[0]
    cand.text_between = " ".join(cand.tokens[start:end])
    return cand

## Candidate PreProcessors

For the purposes of the tutorial, we have three fields (`between_tokens`, `person1_right_tokens`, `person2_right_tokens`) preprocessed in the data, which can be used when creating labeling functions. We also provide the following set of `preprocessor`s for this task in `preprocessors.py`, along with the fields these populate.
* `get_person_text(cand)`: `person_names`
* `get_person_lastnames(cand)`: `person_lastnames`
* `get_left_tokens(cand)`: `person1_left_tokens`, `person2_left_tokens`

In [6]:
from preprocessors import get_left_tokens, get_person_last_names

POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

### LF: basic tokens

In [7]:
from snorkel.labeling import labeling_function

# Check for the `spouse` words appearing between the person mentions
spouses = {
    "spouse", "wife", "husband", "ex-wife", "ex-husband",
    "tied the knot", "tie the knot", "wedding", "married",
}

@labeling_function(resources=dict(spouses=spouses))
def lf_husband_wife(x, spouses):
    return POSITIVE if len(spouses.intersection(set(x.between_tokens))) > 0 else ABSTAIN

In [8]:
# Check for the `spouse` words appearing to the left of the person mentions
@labeling_function(resources=dict(spouses=spouses), pre=[get_left_tokens])
def lf_husband_wife_left_window(x, spouses):
    if len(set(spouses).intersection(set(x.person1_left_tokens))) > 0:
        return POSITIVE
    elif len(set(spouses).intersection(set(x.person2_left_tokens))) > 0:
        return POSITIVE
    else:
        return ABSTAIN

In [9]:
# Check for the person mentions having the same last name
@labeling_function(pre=[get_person_last_names])
def lf_same_last_name(x):
    p1_ln, p2_ln = x.person_lastnames

    if p1_ln and p2_ln and p1_ln == p2_ln:
        return POSITIVE
    return ABSTAIN

In [10]:
# Check for the word `married` between person mentions
@labeling_function()
def lf_married(x):
    return POSITIVE if "married" in x.between_tokens else ABSTAIN

In [11]:
# Check for words that refer to `family` relationships between and to the left of the person mentions
family = {
    "father",
    "mother",
    "sister",
    "sisters",
    "brother",
    "brothers",
    "son",
    "sons",
    "daughter",
    "daughters",
    "grandfather",
    "grandfathers",
    "grandmother",
    "grandmothers",
    "uncle",
    "uncles",
    "aunt",
    "aunts",
    "cousin",
    "cousins",
    "relative",
}
family = family.union({f + "-in-law" for f in family})


@labeling_function(resources=dict(family=family))
def lf_familial_relationship(x, family):
    return NEGATIVE if len(family.intersection(set(x.between_tokens))) > 0 else ABSTAIN


@labeling_function(resources=dict(family=family), pre=[get_left_tokens])
def lf_family_left_window(x, family):
    if len(set(family).intersection(set(x.person1_left_tokens))) > 0:
        return NEGATIVE
    elif len(set(family).intersection(set(x.person2_left_tokens))) > 0:
        return NEGATIVE
    else:
        return ABSTAIN

### LF: EX other relationships

In [12]:
# Check for `other` relationship words between person mentions
other = {
    "boyfriend", "girlfriend", "boss", "employee", "secretary", 
    "co-worker", "friend", "pal", "supervisor", "mentor",
}


@labeling_function(resources=dict(other=other))
def lf_other_relationship(x, other):
    return NEGATIVE if len(other.intersection(set(x.between_tokens))) > 0 else ABSTAIN

### LF: Distant Supervisions

In addition to using factories that encode pattern matching heuristics, we can also write labeling functions that _distantly supervise_ data points. Here, we'll load in a list of known spouse pairs and check to see if the pair of persons in a candidate matches one of these.

[**DBpedia**](http://wiki.dbpedia.org/): Our database of known spouses comes from DBpedia, which is a community-driven resource similar to Wikipedia but for curating structured data. We'll use a preprocessed snapshot as our knowledge base for all labeling function development.

We can look at some of the example entries from DBPedia and use them in a simple distant supervision labeling function.

Make sure `dbpedia.pkl` is in the `spouse/data` directory.

In [13]:
with open("data/dbpedia.pkl", "rb") as f:
    known_spouses = pickle.load(f)

print("* loaded dbpedia %d records" % (len(known_spouses)))
list(known_spouses)[0:5]

* loaded dbpedia 6126 records


[('Diana Lee Inosanto', 'Ron Balicki'),
 ('Karen Gillmor', 'Paul Gillmor'),
 ('Chris Hegedus', 'D. A. Pennebaker'),
 ('Adam Hieronim Sieniawski', 'Katarzyna Kostka'),
 ('Kabir Khan', 'Mini Mathur')]

In [14]:
@labeling_function(resources=dict(known_spouses=known_spouses), pre=[get_person_text])
def lf_distant_supervision(x, known_spouses):
    p1, p2 = x.person_names
    if (p1, p2) in known_spouses or (p2, p1) in known_spouses:
        return POSITIVE
    else:
        return ABSTAIN

In [15]:
from preprocessors import last_name

# Last name pairs for known spouses
last_names = set(
    [
        (last_name(x), last_name(y))
        for x, y in known_spouses
        if last_name(x) and last_name(y)
    ]
)


@labeling_function(resources=dict(last_names=last_names), pre=[get_person_last_names])
def lf_distant_supervision_last_names(x, last_names):
    p1_ln, p2_ln = x.person_lastnames

    return (
        POSITIVE
        if (p1_ln != p2_ln)
        and ((p1_ln, p2_ln) in last_names or (p2_ln, p1_ln) in last_names)
        else ABSTAIN
    )

## Apply Labeling Functions to the Data
We create a list of labeling functions and apply them to the data

In [16]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_husband_wife,
    lf_husband_wife_left_window,
    lf_same_last_name,
    lf_married,
    lf_familial_relationship,
    lf_family_left_window,
    lf_other_relationship,
    lf_distant_supervision,
    lf_distant_supervision_last_names,
]
applier = PandasLFApplier(lfs)

In [17]:
from snorkel.labeling import LFAnalysis

L_dev = applier.apply(df_dev)
# first, get L_train based on raw corpus
L_train = applier.apply(df_train)

100%|██████████| 22254/22254 [00:59<00:00, 372.80it/s]


In [18]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_husband_wife,0,[1],0.110637,0.059765,0.025969,113,198,0.363344
lf_husband_wife_left_window,1,[1],0.036642,0.030238,0.006403,40,63,0.388350
lf_same_last_name,2,[1],0.040555,0.017432,0.009605,19,95,0.166667
lf_married,3,[1],0.019210,0.019210,0.002490,22,32,0.407407
lf_familial_relationship,4,[0],0.128068,0.063678,0.030950,344,16,0.955556
lf_family_left_window,5,[0],0.053718,0.043401,0.011384,148,3,0.980132
lf_other_relationship,6,[0],0.021345,0.006048,0.004980,54,6,0.900000
lf_distant_supervision,7,[1],0.001067,0.001067,0.000000,2,1,0.666667
lf_distant_supervision_last_names,8,[1],0.001067,0.000711,0.000356,0,3,0.000000


### Training the Label Model

Now, we'll train a model of the LFs to estimate their weights and combine their outputs. Once the model is trained, we can combine the outputs of the LFs into a single, noise-aware training label set for our extractor.

In [19]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, Y_dev, n_epochs=5000, log_freq=500, seed=12345)

/Users/m210842/opt/miniconda3/envs/nlpy37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 5000/5000 [00:03<00:00, 1659.32epoch/s]
INFO:root:Finished Training


### Label Model Metrics
Since our dataset is highly unbalanced (91% of the labels are negative), even a trivial baseline that always outputs negative can get a high accuracy. So we evaluate the label model using the F1 score and ROC-AUC rather than accuracy.

In [20]:
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)
print(
    f"Label model f1 score: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')}"
)
print(
    f"Label model roc-auc: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')}"
)

Label model f1 score: 0.4495238095238095
Label model roc-auc: 0.7544328199361433


## Analysis of errors 

In fact, it's better to know why there are so many errors to improve the rules.
After analyzing the errors, you can improve the rule or add new rules to capture more cases.

In [27]:
import numpy as np
import pandas as pd
from typing import Tuple

def filter_labeled_dataframe(
    X: pd.DataFrame, Y: np.ndarray, P: np.ndarray, L: np.ndarray
):
    """Filter out examples covered by any labeling function.
    Parameters
    ----------
    X
        Data points in a Pandas DataFrame.
    Y
        Data points of ground truth.
    P
        Matrix of probabilities output by label model's predict_proba method.
    L
        Matrix of labels emitted by LFs.
    Returns
    -------
    pd.DataFrame
        Data points that were labeled by at least one LF in L.
    np.ndarray
        Probabilities matrix for data points labeled by at least one LF in L.
    """
    mask = (L == -1).all(axis=1)
    return X.iloc[mask], Y[mask], P[mask]

In [28]:
probs_dev = label_model.predict_proba(L_dev)
df_errors, y_errors, p_errors = filter_labeled_dataframe(df_dev, Y_dev, probs_dev, L_dev)

print('* got %d error records in dev' % (len(df_errors)))

* got 2050 error records in dev


In [30]:
from IPython.core.display import display, HTML

cnt = 0
for i in range(0, len(df_errors)):
    r = df_errors.iloc[i]
    y = y_errors[i]
    if y == 0: continue
    
    cnt += 1
    if cnt == 10: break
    
    names = get_person_text(r).person_names
    sent = r['sentence']
    sent = sent.replace(names[0], "<span style='color:red'>%s</span>" % names[0])
    sent = sent.replace(names[1], "<span style='color:green'>%s</span>" % names[1])
    
    display(HTML('* (%s) [%s]-[%s]: %s' % (
        y, names[0], names[1], sent
    )))

For example, in the following case:

```
* (0) [Richards]-[Kathy Hilton]: The Richards are half-sisters to Kathy Hilton, the mother of socialite Paris Hilton and spouse of luxury hotel magnate Richard Howard Hilton.
```

Existing rules cannot capture the "half-sisters" relationship, which results in the deletion of this case in future filtered training set. The will casue the loss in training dataset.

# Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use our noisy training labels to train our end machine learning model. We start by filtering out training data points which did not recieve a label from any LF, as these data points contain no signal.

The `L_train` is the output of LModel based on the input.

The `probs_train` is the probability of `L_train` labels. So, if the labels are the same, the probs are same as well.


In [46]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L_train)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, 
    y=probs_train, 
    L=L_train
)
print('* Size of df_train:', len(df_train))
print('* Size of filtered:', len(df_train_filtered))

* Size of df_train: 22254
* Size of filtered: 6177


In [102]:
L_train[:6, :]

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  1, -1, -1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1, -1, -1]])

In [68]:
probs_train[:6, :]

array([[0.9324084 , 0.0675916 ],
       [0.9324084 , 0.0675916 ],
       [0.9324084 , 0.0675916 ],
       [0.29667719, 0.70332281],
       [0.42529014, 0.57470986],
       [0.02212763, 0.97787237]])

Next, we train a simple [LSTM](https://en.wikipedia.org/wiki/Long_short-term_memory) network for classifying candidates. `tf_model` contains functions for processing features and building the keras model for training and evaluation.

## Try with TensorFlow

In [48]:
import tensorflow as tf
from tf_model import get_feature_arrays
from utils import get_n_epochs

from tensorflow.keras.layers import (
    Bidirectional,
    Concatenate,
    Dense,
    Embedding,
    Input,
    LSTM,
)

def bilstm(
    tokens: tf.Tensor,
    rnn_state_size: int = 64,
    num_buckets: int = 40000,
    embed_dim: int = 36,
):
    ids = tf.strings.to_hash_bucket(tokens, num_buckets)
    embedded_input = Embedding(num_buckets, embed_dim)(ids)
    return Bidirectional(LSTM(rnn_state_size, activation=tf.nn.relu))(
        embedded_input, mask=tf.strings.length(tokens)
    )

def get_model(
    rnn_state_size: int = 64, 
    num_buckets: int = 40000, 
    embed_dim: int = 12,
    learning_rate: float = 0.001
) -> tf.keras.Model:
    """
    Return LSTM model for predicting label probabilities.
    Args:
        rnn_state_size: LSTM state size.
        num_buckets: Number of buckets to hash strings to integers.
        embed_dim: Size of token embeddings.
    Returns:
        model: A compiled LSTM model.
    """
    left_ph = Input((None,), dtype="string")
    bet_ph = Input((None,), dtype="string")
    right_ph = Input((None,), dtype="string")
    left_embs = bilstm(left_ph, rnn_state_size, num_buckets, embed_dim)
    bet_embs = bilstm(bet_ph, rnn_state_size, num_buckets, embed_dim)
    right_embs = bilstm(right_ph, rnn_state_size, num_buckets, embed_dim)
    layer = Concatenate(1)([left_embs, bet_embs, right_embs])
    layer = Dense(64, activation=tf.nn.relu)(layer)
    layer = Dense(32, activation=tf.nn.relu)(layer)
    probabilities = Dense(2, activation=tf.nn.softmax)(layer)
    model = tf.keras.Model(inputs=[bet_ph, left_ph, right_ph], outputs=probabilities)
    model.compile(tf.optimizers.Adam(learning_rate), "categorical_crossentropy")
    return model

### Test with dev df

In [49]:
model3 = get_model()
batch_size = 64
epochs = 10

X_dev = get_feature_arrays(df_dev)
probs_dev = label_model.predict_proba(L_dev)

model3.fit(X_dev, probs_dev, batch_size=batch_size, epochs=epochs)

Epoch 1/20
44/44 [==============================] - 11s 66ms/step - loss: 0.5132
Epoch 2/20
44/44 [==============================] - 3s 77ms/step - loss: 0.4190
Epoch 3/20
44/44 [==============================] - 3s 74ms/step - loss: 0.3834
Epoch 4/20
44/44 [==============================] - 3s 79ms/step - loss: 0.3486
Epoch 5/20
44/44 [==============================] - 4s 80ms/step - loss: 0.3220
Epoch 6/20
44/44 [==============================] - 4s 83ms/step - loss: 0.3122
Epoch 7/20
44/44 [==============================] - 3s 78ms/step - loss: 0.3071
Epoch 8/20
44/44 [==============================] - 3s 77ms/step - loss: 0.3039
Epoch 9/20
44/44 [==============================] - 3s 77ms/step - loss: 0.3019
Epoch 10/20
44/44 [==============================] - 3s 74ms/step - loss: 0.3011
Epoch 11/20
44/44 [==============================] - 3s 78ms/step - loss: 0.3008
Epoch 12/20
44/44 [==============================] - 4s 82ms/step - loss: 0.2999
Epoch 13/20
44/44 [=================

In [50]:
X_test = get_feature_arrays(df_test)
probs_test3 = model3.predict(X_test)
preds_test3 = probs_to_preds(probs_test3)
print(
    f"Test F1 when trained with soft labels: {metric_score(Y_test, preds=preds_test3, metric='f1')}"
)
print(
    f"Test ROC-AUC when trained with soft labels: {metric_score(Y_test, probs=probs_test3, metric='roc_auc')}"
)

85/85 [==============================] - 2s 9ms/step
Test F1 when trained with soft labels: 0.3414634146341463
Test ROC-AUC when trained with soft labels: 0.7149626635432869


### Test with filtered df

In [51]:
X_train = get_feature_arrays(df_train_filtered)
model = get_model()
batch_size = 64
model.fit(X_train, probs_train_filtered, batch_size=batch_size, epochs=10)

Epoch 1/10
97/97 [==============================] - 14s 63ms/step - loss: 0.6500
Epoch 2/10
97/97 [==============================] - 7s 70ms/step - loss: 0.5194
Epoch 3/10
97/97 [==============================] - 7s 76ms/step - loss: 0.4816
Epoch 4/10
97/97 [==============================] - 9s 89ms/step - loss: 0.4651
Epoch 5/10
97/97 [==============================] - 10s 105ms/step - loss: 0.4557
Epoch 6/10
97/97 [==============================] - 8s 79ms/step - loss: 0.4517
Epoch 7/10
97/97 [==============================] - 7s 76ms/step - loss: 0.4494
Epoch 8/10
97/97 [==============================] - 7s 72ms/step - loss: 0.4482
Epoch 9/10
97/97 [==============================] - 7s 72ms/step - loss: 0.4475
Epoch 10/10
97/97 [==============================] - 7s 71ms/step - loss: 0.4469


Finally, we evaluate the trained model by measuring its F1 score and ROC_AUC.

In [52]:
X_test = get_feature_arrays(df_test)
probs_test = model.predict(X_test)
preds_test = probs_to_preds(probs_test)
print(
    f"Test F1 when trained with soft labels: {metric_score(Y_test, preds=preds_test, metric='f1')}"
)
print(
    f"Test ROC-AUC when trained with soft labels: {metric_score(Y_test, probs=probs_test, metric='roc_auc')}"
)

85/85 [==============================] - 2s 9ms/step
Test F1 when trained with soft labels: 0.39143730886850153
Test ROC-AUC when trained with soft labels: 0.7650759106880919


### Test with ALL df

In [53]:
X_train = get_feature_arrays(df_train)
model2 = get_model()
batch_size = 64
epochs = 10
model2.fit(X_train, probs_train, batch_size=batch_size, epochs=epochs)

Epoch 1/20
348/348 [==============================] - 35s 76ms/step - loss: 0.4021
Epoch 2/20
348/348 [==============================] - 27s 78ms/step - loss: 0.3263
Epoch 3/20
348/348 [==============================] - 27s 78ms/step - loss: 0.3173
Epoch 4/20
348/348 [==============================] - 27s 78ms/step - loss: 0.3115
Epoch 5/20
348/348 [==============================] - 27s 77ms/step - loss: 0.3082
Epoch 6/20
348/348 [==============================] - 28s 80ms/step - loss: 0.3061
Epoch 7/20
348/348 [==============================] - 28s 80ms/step - loss: 0.3053
Epoch 8/20
348/348 [==============================] - 27s 78ms/step - loss: 0.3045
Epoch 9/20
348/348 [==============================] - 29s 82ms/step - loss: 0.3041
Epoch 10/20
348/348 [==============================] - 27s 78ms/step - loss: 0.3040
Epoch 11/20
348/348 [==============================] - 28s 81ms/step - loss: 0.3036
Epoch 12/20
348/348 [==============================] - 27s 79ms/step - loss: 0.3033
E

In [54]:
X_test = get_feature_arrays(df_test)
probs_test2 = model2.predict(X_test)
preds_test2 = probs_to_preds(probs_test2)
print(
    f"Test F1 when trained with soft labels: {metric_score(Y_test, preds=preds_test2, metric='f1')}"
)
print(
    f"Test ROC-AUC when trained with soft labels: {metric_score(Y_test, probs=probs_test2, metric='roc_auc')}"
)

85/85 [==============================] - 2s 9ms/step
Test F1 when trained with soft labels: 0.4810126582278481
Test ROC-AUC when trained with soft labels: 0.7758168389082458


# Application

In [77]:
df_case = df_train.sample(5)
X_case = get_feature_arrays(df_case)
y_case = model2.predict(X_case)

for i in range(0, len(df_case)):
    r = df_case.iloc[i]
    y = y_case[i][1] * 100
    
    names = get_person_text(r).person_names
    sent = r['sentence']
    sent = sent.replace(names[0], "<span style='color:red'>%s</span>" % names[0])
    sent = sent.replace(names[1], "<span style='color:green'>%s</span>" % names[1])
    
    display(HTML('* (%.2f%% spouse) [%s]-[%s]: %s' % (
        y, names[0], names[1], sent
    )))

1/1 [==============================] - 0s 29ms/step


# Summary
>   In this tutorial, we showed how Snorkel can be used for Information Extraction. We demonstrated how to create LFs that leverage keywords and external knowledge bases (distant supervision). Finally, we showed how a model trained using the probabilistic outputs of the Label Model can achieve comparable performance while generalizing to all data points.

TLDR
1. Small dataset (df_dev) works but low performance
2. Filtered generated dataset (filtered df_train) works better and trains quickly
3. All generated dataset (all df_train) works best but long time to train and use
